# Introduction

In this tutorial series, the `RobotKinematics` package will be introduced. This package was build for the portfolio of me, Matthias Hamacher. It uses `sympy` to calculate the kinematics symbolically. 

In this instance of the series, I will introduce how a robot can be defined and how the custom matrix and vector classes work.

# Turorial

In [1]:
from sympy import symbols, pi

from robot import Robot, JointType

from math_utils import Vector, Matrix

## 1) Variable Initialization

First, we need to define the variables that we will later use to define our robot. Generally, each robot has at least one control variable per link. There may be additional variables that define the length between certain links. 

Since the code for the kinematics calculations creates own variables for the derivative, it is beneficial to use a common convention for the naming of the variables. Each prismatic joint is assinged a variable di and each rotational the variable ti (short for thetai). The i is substituted for the number of the joint, e.g. a RPRP robot has the variables t1, d2, t3, and d4.

In [2]:
d1, d2, d3, d4 = symbols('d1 d2 d3 d4')
t1, t2, t3, t4 = symbols('t1 t2 t3 t4')
l1, l2, l3, l4 = symbols('l1 l2 l3 l4')

## 2) Robot Initialization

Now, we can define the robot with the previously intialized variables. A robot is initialized with a list of joints. Since it is annoying to define each joint individually, the `Robot.from_dh_parameters()` function will be used to build a robot from DH parameters. 

To define the DH-parameters of an $N$-joint robot, an matrix with four columns and $N+1$ rows has to be defined. The last row is needed for the end-effector frame. If the end-effector coincides with the last joint frame, the last row can all be zeros.

Each row $i$ has to be filled with the following parameters: [$a_{i-1}$, $\alpha_{i-1}$, $d_i$, $\theta_i$]. 
If parameters in a row are set to defined angles, be sure to use radians 

Below we define a 3R robot. 

In [3]:
dh = [
    [0, 0, 0, t1],
    [l1, pi/2, 0, t2],
    [l2,0,0,t3],
    [l3, 0, 0, 0]
]

After having defined the DH-parameters, we can simply call the `Robot.from_dh_parameters()` function. This function needs two other parameters: a list of joint types, and a list of control variables. 

The control variables are not used in the current implimentation and can be set to `None`. The list of joint types are needed. There are four joint types: `BASE`, `PRISMATIC`, `ROTATIONAL`, and `END_EFFECTOR`. Each robot automatically has a `BASE` joint as the first joint. This joint has no rotation, translation, or other parameters and serves as the world coordinate frame. The `END_EFFECTOR` is similar, but at the end. If you want to have a end-effector frame, add a last row in the dh parameters without any control variables (in the example above that has been done). 

Since the base joint and the end-effector are automatically defined, only the prismatic and rotational joint types have to be set

In [5]:
robot = Robot.from_dh_parameters(dh, [JointType.ROTATIONAL, JointType.ROTATIONAL, JointType.ROTATIONAL])

print(robot)

Robot[
 - BASE joint	 [a: 0.0, alpha: 0.0, d: 0, theta: 0]
 - ROTATIONAL joint	 [a: 0, alpha: 0, d: 0, theta: t1]
 - ROTATIONAL joint	 [a: l1, alpha: pi/2, d: 0, theta: t2]
 - ROTATIONAL joint	 [a: l2, alpha: 0, d: 0, theta: t3]
 - END_EFFECTOR	 [a: l3, alpha: 0, d: 0, theta: 0]
]


It quickly becomes quite annoying to set the joint types in such a way. Therefore, two simplifications can be done. 

Since all joints are of the same type in our example, we can only pass the joint type to the function and it will work the same way.

In [6]:
robot = Robot.from_dh_parameters(dh, JointType.ROTATIONAL)

print(robot)

Robot[
 - BASE joint	 [a: 0.0, alpha: 0.0, d: 0, theta: 0]
 - ROTATIONAL joint	 [a: 0, alpha: 0, d: 0, theta: t1]
 - ROTATIONAL joint	 [a: l1, alpha: pi/2, d: 0, theta: t2]
 - ROTATIONAL joint	 [a: l2, alpha: 0, d: 0, theta: t3]
 - END_EFFECTOR	 [a: l3, alpha: 0, d: 0, theta: 0]
]


As you can see, the two robots are exactly the same. 

While this works well for robots with a uniform joint type, it does not work for other more complex robots. There is another simplification that works well for every robot. We can also define the types with a string. 

In [7]:
robot = Robot.from_dh_parameters(dh, 'RRR')

print(robot)

Robot[
 - BASE joint	 [a: 0.0, alpha: 0.0, d: 0, theta: 0]
 - ROTATIONAL joint	 [a: 0, alpha: 0, d: 0, theta: t1]
 - ROTATIONAL joint	 [a: l1, alpha: pi/2, d: 0, theta: t2]
 - ROTATIONAL joint	 [a: l2, alpha: 0, d: 0, theta: t3]
 - END_EFFECTOR	 [a: l3, alpha: 0, d: 0, theta: 0]
]


Each rotational joint is specified by an `R` and a prismatic joint by a `P` (the capitalization is not important). We can also use numbers followed by a letter to describe multiple joints of the same type in a row. 

In [8]:
robot = Robot.from_dh_parameters(dh, '3r')

print(robot)

Robot[
 - BASE joint	 [a: 0.0, alpha: 0.0, d: 0, theta: 0]
 - ROTATIONAL joint	 [a: 0, alpha: 0, d: 0, theta: t1]
 - ROTATIONAL joint	 [a: l1, alpha: pi/2, d: 0, theta: t2]
 - ROTATIONAL joint	 [a: l2, alpha: 0, d: 0, theta: t3]
 - END_EFFECTOR	 [a: l3, alpha: 0, d: 0, theta: 0]
]


## 3) Matrices and Vectors

TODO